In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_excel('TP_pivot_JWBLACK.xlsx')

In [3]:
df2 = df1.loc[df1['Tipo de tienda'] == 'CONVENIENCIA']

In [4]:
df3 = df2[['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer','CADENA',
'ruta','Area Manager','Area Executive','Touch Point']]

In [5]:
weeks = [22]

In [6]:
weeks_headers = ['week22'] #length must be the same as previous line

In [7]:
df4 = df3[df3['SEMANA'].isin(weeks)]

In [8]:
df5 = df4.groupby(['Store Number','SEMANA','City','State','Tipo de tienda','Region','Retailer','CADENA','ruta',
'Area Manager','Area Executive']).sum().reset_index()

In [9]:
rutas_vdm = ['DIA.PROM.08','DIA.PROM.09','DIA.PROM.11','DIA.PROM.12','DIA.PROM.13']

In [10]:
df5['Total Score'] = np.where(df5['ruta'].isin(rutas_vdm), df5['Touch Point']*2, df5['Touch Point']) #tested good

In [13]:
df_score = df5[['Store Number','ruta','SEMANA','Total Score']]

In [14]:
df_score_ruta = df_score.groupby(['ruta','SEMANA']).agg({
'Store Number': 'count',
'Total Score': 'sum'
}).reset_index()

In [15]:
df_score_ruta = df_score_ruta[['ruta','SEMANA','Total Score']]

In [16]:
pivot_weekly_score = pd.pivot_table(df_score_ruta,
index=['ruta'],
columns=['SEMANA'], aggfunc='sum', fill_value=0).reset_index()

In [17]:
pivot_weekly_score.columns = pivot_weekly_score.columns.get_level_values(0)

In [18]:
pivot_weekly_score.columns = pivot_weekly_score.columns[:1].tolist() + weeks_headers

In [19]:
df_leaderboard = pivot_weekly_score

In [21]:
df_leaderboard['Final Score'] = df_leaderboard.sum(axis=1, numeric_only=True)

In [22]:
df_leaderboard = df_leaderboard.sort_values(by='Final Score', ascending=False)

In [23]:
df_leaderboard['Lugar'] = np.arange(1,len(df_leaderboard)+1)

In [24]:
df_leaderboard.head() #portada del sitio web

,ruta,week22,Final Score,Lugar
12,DIA.PROM.13,382,382,1
10,DIA.PROM.11,382,382,2
22,DIA.PROM.23,313,313,3
7,DIA.PROM.08,278,278,4
21,DIA.PROM.22,246,246,5


In [25]:
df_score.head()

,Store Number,ruta,SEMANA,Total Score
0,4588,DIA.PROM.11,22,6
1,4620,DIA.PROM.11,22,6
2,4621,DIA.PROM.08,22,2
3,4648,DIA.PROM.11,22,6
4,4649,DIA.PROM.11,22,6


In [26]:
df_leaderboard = df_leaderboard.rename(columns={
'Final Score': 'fs'
})

In [27]:
df_score = df_score.rename(columns={
'Store Number': 'store',
'SEMANA': 'week',
'Total Score': 'score'
})

In [28]:
df2_store_names = df2[['Store Number','Store Name']]

In [29]:
df2_store_names = df2_store_names.rename(columns={
'Store Number': 'store',
'Store Name': 'name'
})

In [30]:
df2_store_names = df2_store_names.drop_duplicates(subset='store')

In [31]:
df_score = df_score.merge(df2_store_names, how='left')

In [33]:
df_score = df_score[['name','ruta','week','score']]

In [35]:
df_leaderboard.to_json('leaderboard_conv.js', orient='records')

In [36]:
df_score.to_json('score_conv.js', orient='records')